# CH. 9 - Hotspot Analysis
## Activities

#### Activity 9.01: One Dimensional Density Estimation

In [ ]:
get_ipython().run_line_magic('matplotlib', 'inline')

import matplotlib.pyplot as plt
import numpy
import pandas
import seaborn
import sklearn.model_selection
import sklearn.neighbors

seaborn.set()

In [ ]:
# randomly generate values
# for which we will estimate a density

rand = numpy.random.RandomState(100)
vals = rand.randn(1000)  # standard normal
vals[375:] += 3.5

In [ ]:
# plot the randomly generated values
# as a histogram

fig, ax = plt.subplots(figsize=(14, 10))
ax.hist(vals, bins=50, density=True, label='Sampled Values')
ax.plot(vals, -0.005 - 0.01 * numpy.random.random(len(vals)), '+k', label='Individual Points')
ax.legend(loc='upper right')
plt.show()

In [ ]:
# grid search
# optimal bandwidth value

bandwidths = 10 ** numpy.linspace(-1, 1, 100)

grid = sklearn.model_selection.GridSearchCV(
    estimator=sklearn.neighbors.KernelDensity(kernel="gaussian"),
    param_grid={"bandwidth": bandwidths},
    cv=10
)
grid.fit(vals[:, None])

In [ ]:
# extract the optimal bandwidth value

best_bandwidth = grid.best_params_["bandwidth"]

print(
    "Best Bandwidth Value: {}"
    .format(best_bandwidth)
)

In [ ]:
# replot the histogram
# with the optimal estimated density overlayed

fig, ax = plt.subplots(figsize=(14, 10))

ax.hist(vals, bins=50, density=True, alpha=0.75, label='Sampled Values')

x_vec = numpy.linspace(-4, 8, 10000)[:, numpy.newaxis]
log_density = numpy.exp(grid.best_estimator_.score_samples(x_vec))
ax.plot(x_vec[:, 0], log_density, '-', linewidth=4, label='Kernel = Gaussian')

ax.legend(loc='upper right')
plt.show()

In [ ]:
# Activity 9.01 Unit Test

def unittest_activity_9_01(d, b):
    assert int(round(numpy.sum(d))) == 2171, "Vals vector sum incorrect"
    assert round(b, 1) == 0.4, "Wrong bandwidth"

unittest_activity_9_01(d=vals, b=best_bandwidth)

#### Activity 9.02: London Crime

In [ ]:
# define base path

base_path = "./metro-jul18-dec18/{yr_mon}/{yr_mon}-metropolitan-street.csv"
print(base_path)

In [ ]:
# create the six file tags
# used to load in the data

yearmon_list = [
    "2018-0" + str(i) if i <= 9 else "2018-" + str(i) 
    for i in range(7, 13)
]

print(yearmon_list)

In [ ]:
# load the data
# print some basic information about the data

data_yearmon_list = []

for idx, i in enumerate(yearmon_list):
    df = pandas.read_csv(
        base_path.format(yr_mon=i), 
        header=0
    )
    
    data_yearmon_list.append(df)
    
    if idx == 0:
        print("Month: {}".format(i))
        print("Dimensions: {}".format(df.shape))
        print("Head:\n{}\n".format(df.head(2)))

In [ ]:
london = pandas.concat(data_yearmon_list)

In [ ]:
# perform diagnostics on the
# full concatenated data set

print(
    "Dimensions - Full Data:\n{}\n"
    .format(london.shape)
)
print(
    "Unique Months - Full Data:\n{}\n"
    .format(london["Month"].unique())
)
print(
    "Number of Unique Crime Types - Full Data:\n{}\n"
    .format(london["Crime type"].nunique())
)
print(
    "Unique Crime Types - Full Data:\n{}\n"
    .format(london["Crime type"].unique())
)
print(
    "Count Occurrences Of Each Unique Crime Type - Full Type:\n{}\n"
    .format(london["Crime type"].value_counts())
)

In [ ]:
# filter the data down

london_subset = london[["Month", "Longitude", "Latitude", "Crime type"]]
london_subset.head(5)

In [ ]:
# plotting bicycle theft
# for three randomly selected months

crime_bicycle_jul = london_subset[
    (london_subset["Crime type"] == "Bicycle theft") & 
    (london_subset["Month"] == "2018-07")
]

seaborn.jointplot("Longitude", "Latitude", crime_bicycle_jul, kind="kde")

In [ ]:
crime_bicycle_sept = london_subset[
    (london_subset["Crime type"] == "Bicycle theft") & 
    (london_subset["Month"] == "2018-09")
]

seaborn.jointplot("Longitude", "Latitude", crime_bicycle_sept, kind="kde")

In [ ]:
crime_bicycle_dec = london_subset[
    (london_subset["Crime type"] == "Bicycle theft") & 
    (london_subset["Month"] == "2018-12")
]

seaborn.jointplot("Longitude", "Latitude", crime_bicycle_dec, kind="kde")

In [ ]:
# plotting shoplifting
# for three randomly selected months

crime_shoplift_aug = london_subset[
    (london_subset["Crime type"] == "Shoplifting") & 
    (london_subset["Month"] == "2018-08")
]

seaborn.jointplot("Longitude", "Latitude", crime_shoplift_aug, kind="kde")

In [ ]:
crime_shoplift_oct = london_subset[
    (london_subset["Crime type"] == "Shoplifting") & 
    (london_subset["Month"] == "2018-10")
]

seaborn.jointplot("Longitude", "Latitude", crime_shoplift_oct, kind="kde")

In [ ]:
crime_shoplift_nov = london_subset[
    (london_subset["Crime type"] == "Shoplifting") & 
    (london_subset["Month"] == "2018-11")
]

seaborn.jointplot("Longitude", "Latitude", crime_shoplift_nov, kind="kde")

In [ ]:
# plotting burglary
# for three randomly selected months

crime_burglary_jul = london_subset[
    (london_subset["Crime type"] == "Burglary") & 
    (london_subset["Month"] == "2018-07")
]

seaborn.jointplot("Longitude", "Latitude", crime_burglary_jul, kind="kde")

In [ ]:
crime_burglary_oct = london_subset[
    (london_subset["Crime type"] == "Burglary") & 
    (london_subset["Month"] == "2018-10")
]

seaborn.jointplot("Longitude", "Latitude", crime_burglary_oct, kind="kde")

In [ ]:
crime_burglary_dec = london_subset[
    (london_subset["Crime type"] == "Burglary") & 
    (london_subset["Month"] == "2018-12")
]

seaborn.jointplot("Longitude", "Latitude", crime_burglary_dec, kind="kde")

In [ ]:
# Activity 9.02 Unit Test

def unittest_activity_9_02(data, env, tag):
    assert data.shape == (546032, 12), "London data frame dimensions wrong"
    tag_length = int(len([i for i in env if i.startswith(tag)]))
    assert tag_length == 9, "Filtered data frame count incorrect"

unittest_activity_9_02(data=london, env=dir(), tag='crime')